In [1]:
!pip install llama-index
!pip install chromadb
!pip install llama-index-vector-stores-chroma
%mkdir data
!pip install llama-index-embeddings-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.3/251.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.9 MB/s eta 0

In [6]:
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

documents = SimpleDirectoryReader("./data").load_data()

# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./chroma_db")

# create collection
chroma_collection = db.get_or_create_collection("mifos")

# select embedding model
embed_model = HuggingFaceEmbedding(
    model_name="nomic-ai/nomic-embed-text-v2-moe",
    trust_remote_code=True
)

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    embed_model=embed_model
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/554 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.41k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [9]:
from llama_index.llms.openai import OpenAI
query_engine = index.as_query_engine()
llm = OpenAI(model="gpt-4o")
from IPython.display import Markdown, display

chat_engine = index.as_chat_engine(
    chat_mode = "best",
    llm=llm,
    embed_model=embed_model,
    similarity_top_k=5,
    system_prompt="You are a helpful assistant that provides accurate information based on the documents provided.",
    verbose=False
)

In [10]:
response_chat = chat_engine.chat("What kind of projects are participants looking for in GSOC 2025?")
display(Markdown(f"<b>{response_chat}</b>"))

<b>Participants in GSOC 2025 are interested in projects related to AI and machine learning, such as Bank Statement Analysis and Voice-Driven Banking via Large Acoustic Models. They are also keen on projects involving backend development, microservices, scalability, and fintech applications, like the Online Banking App 5.0 - Customer Loan Management Portal. Overall, participants are looking for opportunities to apply their skills in AI, backend development, and fintech to contribute to impactful open-source projects.</b>

In [11]:
response_chat = chat_engine.chat("How can I apply for Gsoc?")
display(Markdown(f"<b>{response_chat}</b>"))

<b>To apply for Google Summer of Code (GSoC), follow these steps:

1. **Visit the GSoC Website**: Start by visiting the official GSoC website to understand the program's requirements and timeline.

2. **Explore Participating Organizations**: Look through the list of participating organizations and their project ideas to find one that matches your skills and interests.

3. **Engage with the Community**: Join the organization's community channels, contribute to discussions, and familiarize yourself with their codebase.

4. **Prepare a Project Proposal**: Draft a strong proposal that outlines your project plan, timeline, and how you intend to contribute to the organization.

5. **Submit Your Proposal**: Submit your proposal through the GSoC website during the application period.

6. **Seek Feedback**: Before finalizing your proposal, seek feedback from mentors or community members to improve it.

By following these steps, you can effectively apply for GSoC.</b>

In [12]:
response_chat = chat_engine.chat("What are the mentor's suggestion for Gsoc applicants?")
display(Markdown(f"<b>{response_chat}</b>"))

<b>Mentors suggest that GSoC applicants should:

1. **Explore Project Repositories**: Understand the codebase and ongoing issues by exploring the project repositories in detail.

2. **Start with Small Contributions**: Begin with small contributions, like fixing bugs or addressing open issues, to get familiar with the project workflow.

3. **Join Community Channels**: Connect with the community by joining relevant channels to stay updated and engage with other contributors.

4. **Seek Guidance**: Don't hesitate to ask mentors for clarification or help, as they are generally supportive and willing to provide guidance.

By following these suggestions, applicants can better prepare themselves for a successful GSoC experience.</b>